## Random Forest Regression
#### Resources: https://www.geeksforgeeks.org/random-forest-regression-in-python/

In [1]:
# import libraries
import pandas as pd
import warnings 

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine #, func
from sqlalchemy.orm import Session
from sqlalchemy import text

warnings.filterwarnings('ignore')

In [2]:
#Read data and create dataframe

filtered_df = pd.read_csv("Resources/filtered_df.csv")
#filtered_df

# Create a SQLite database engine
engine = create_engine('sqlite:///filtered_df.sqlite')

# reflect an existing database into a new model
Base=automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)

# Write DataFrame to SQLite database
filtered_df.to_sql('realtor_filtered', con=engine, if_exists='replace', index=False)

# Confirm the data has been written by querying the database
query = "SELECT * FROM realtor_filtered;"  # Example query
with engine.connect() as conn:
    result = conn.execute(text(query))

In [3]:
df = result
df = pd.DataFrame(df)
df

,status,price,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date
0,for_sale,180000.0,2.0,1.0,0.34,Agawam,Massachusetts,1001.0,676.0,None
1,for_sale,239900.0,3.0,1.0,0.46,Agawam,Massachusetts,1001.0,1196.0,None
2,for_sale,525000.0,3.0,3.0,0.45,Agawam,Massachusetts,1001.0,2314.0,2014-06-25
3,for_sale,289900.0,3.0,2.0,0.36,Agawam,Massachusetts,1001.0,1276.0,2012-10-12
4,for_sale,275000.0,4.0,2.0,0.11,Agawam,Massachusetts,1001.0,1732.0,None
...,...,...,...,...,...,...,...,...,...,...
1353428,sold,359900.0,4.0,2.0,0.33,Richland,Washington,99354.0,3600.0,2022-03-25
1353429,sold,350000.0,3.0,2.0,0.10,Richland,Washington,99354.0,1616.0,2022-03-25
1353430,sold,440000.0,6.0,3.0,0.50,Richland,Washington,99354.0,3200.0,2022-03-24
1353431,sold,179900.0,2.0,1.0,0.09,Richland,Washington,99354.0,933.0,2022-03-24


In [4]:
# reorder the columns
df = df[['status','bed', 'bath', 'acre_lot', 'city','state','zip_code','house_size', 'price']]
df

,status,bed,bath,acre_lot,city,state,zip_code,house_size,price
0,for_sale,2.0,1.0,0.34,Agawam,Massachusetts,1001.0,676.0,180000.0
1,for_sale,3.0,1.0,0.46,Agawam,Massachusetts,1001.0,1196.0,239900.0
2,for_sale,3.0,3.0,0.45,Agawam,Massachusetts,1001.0,2314.0,525000.0
3,for_sale,3.0,2.0,0.36,Agawam,Massachusetts,1001.0,1276.0,289900.0
4,for_sale,4.0,2.0,0.11,Agawam,Massachusetts,1001.0,1732.0,275000.0
...,...,...,...,...,...,...,...,...,...
1353428,sold,4.0,2.0,0.33,Richland,Washington,99354.0,3600.0,359900.0
1353429,sold,3.0,2.0,0.10,Richland,Washington,99354.0,1616.0,350000.0
1353430,sold,6.0,3.0,0.50,Richland,Washington,99354.0,3200.0,440000.0
1353431,sold,2.0,1.0,0.09,Richland,Washington,99354.0,933.0,179900.0


In [5]:
# Set the features and the target
X = df.iloc[:,1:8]
y = df.iloc[:,8]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1353433 entries, 0 to 1353432
Data columns (total 9 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   status      1353433 non-null  object 
 1   bed         1353433 non-null  float64
 2   bath        1353433 non-null  float64
 3   acre_lot    1353433 non-null  float64
 4   city        1353433 non-null  object 
 5   state       1353433 non-null  object 
 6   zip_code    1353433 non-null  float64
 7   house_size  1353433 non-null  float64
 8   price       1353433 non-null  float64
dtypes: float64(6), object(3)
memory usage: 92.9+ MB


In [7]:
# separate string entries from numerical entries
label_encoder = LabelEncoder()
x_categorical = df.select_dtypes(include=['object']).apply(label_encoder.fit_transform)
x_numerical = df.select_dtypes(exclude=['object']).values
x = pd.concat([pd.DataFrame(x_numerical), x_categorical], axis=1).values
 
# define and fit the model
regressor = RandomForestRegressor(n_estimators=25, random_state=0, oob_score=True)
regressor.fit(x, y)

RandomForestRegressor(n_estimators=25, oob_score=True, random_state=0)

In [8]:
# Evaluating the model
from sklearn.metrics import mean_squared_error, r2_score
 
# Access the Out of Bag Score
oob_score = regressor.oob_score_
print(f'Out-of-Bag Score: {oob_score}')
 
# Make predictions
predictions = regressor.predict(x)
 
# Evaluate the model
mse = mean_squared_error(y, predictions)
print(f'Mean Squared Error: {mse}')

# Determine the R-Squared Value
r2 = r2_score(y, predictions)
print(f'R-squared: {r2}')


Out-of-Bag Score: 0.9276915301692188
Mean Squared Error: 13580310530.126436
R-squared: 0.9908399469832393
